<a href="https://colab.research.google.com/github/annie-lyu/word-embedding/blob/master/ldaseq_csStack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from gensim import corpora
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
import numpy
from gensim.matutils import hellinger
import nltk


In [3]:
import glob
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
## preparation for text preprocessing
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
stop_words = list(set(stopwords.words('english') + list(string.ascii_lowercase)))
stop_words = stop_words + ["would", "could", "use", "also", "might", "one", "whether", "like", "first", "code", "given"]
stop_words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['does',
 'here',
 'hasn',
 'wouldn',
 'have',
 'p',
 'there',
 'how',
 'hers',
 "won't",
 'shan',
 'in',
 'we',
 'because',
 'z',
 'other',
 'j',
 'theirs',
 'yourself',
 'most',
 't',
 'am',
 'if',
 "you'd",
 'and',
 "don't",
 'are',
 've',
 'this',
 'b',
 "doesn't",
 "you've",
 'now',
 'll',
 'then',
 'don',
 'aren',
 'can',
 "couldn't",
 'ain',
 'he',
 'were',
 'ours',
 'over',
 'or',
 'n',
 'me',
 'himself',
 "mustn't",
 'itself',
 'too',
 "she's",
 "didn't",
 'to',
 'x',
 'i',
 'their',
 'very',
 'mustn',
 'mightn',
 'no',
 'be',
 'which',
 'who',
 "that'll",
 'until',
 'd',
 'e',
 'into',
 'as',
 'same',
 'it',
 'from',
 'up',
 'own',
 "shan't",
 "weren't",
 'r',
 'of',
 'f',
 'for',
 'any',
 'was',
 "it's",
 'she',
 'being',
 "aren't",
 'off',
 'what',
 "needn't",
 'again',
 'l',
 'is',
 'at',
 'by',
 'ma',
 'w',
 'they',
 'on',
 'such',
 'nor',
 'those',
 'more',
 'his',
 "mightn't",
 'some',
 'didn',
 'c',
 "shouldn't",
 'couldn',
 'them',
 'will',
 'haven',
 'our',
 'shouldn

In [5]:
import re
def removePunctuation(text):
    punctuation = '\\\\!,;:?"\'.-=^/|+-()'
    text = re.sub(r'[{}]+'.format(punctuation),'',text)
    text = text.replace("\n", "")
    return text

In [6]:
lines =  [line.lower() for line in open("/content/drive/My Drive/words/cs-stack.txt",'r', errors='ignore')]
posts = [removePunctuation(line) for line in lines]
posts[1]


OSError: ignored

In [ ]:
lines =  open("/content/drive/My Drive/words/cs-stack.txt",'r', errors='ignore').read()


In [92]:
## establish dictionary and remove stop words
dictionary_cs= corpora.Dictionary(post.split() for post in posts)
stop_ids = [dictionary_cs.token2id[stopword] for stopword in stop_words if stopword in dictionary_cs.token2id]
dictionary_cs.filter_tokens(stop_ids)
dictionary_cs.compactify()
dictionary_cs.save("/content/drive/My Drive/words/stack_dictionary")


## create vocabulary file
class MyCorpus(object):
    def __iter__(self):
        for line in open("/content/drive/My Drive/words/cs-stack.txt", errors= "ignore"):
            yield dictionary_cs.doc2bow(line.lower().split())

corpus_memory_friendly_cs = MyCorpus()
corpus_cs = [vector for vector in corpus_memory_friendly_cs]
corpora.BleiCorpus.serialize('/content/drive/My Drive/words/corpus_cs', corpus_cs)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [96]:
## loading and setting
try:
  dictionary_cs = Dictionary.load("/content/drive/My Drive/words/stack_dictionary")
except FileNotFoundError as e:
  raise ValueError("SKIP: Please download the Corpus/news_dictionary dataset.")
corpus_cs = bleicorpus.BleiCorpus('/content/drive/My Drive/words/corpus_cs')

time_slice = [6000]



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [97]:
ldaseq_cs_5 = ldaseqmodel.LdaSeqModel(corpus=corpus_cs, id2word=dictionary_cs, time_slice = time_slice, num_topics=5)
ldaseq_cs_10 = ldaseqmodel.LdaSeqModel(corpus=corpus_cs, id2word=dictionary_cs, time_slice = time_slice, num_topics=10)



/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/gensim/models/ldaseqmodel.py:1472: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.fabs((lhood_old - lhood) / (lhood_old * total))
/usr/local/lib/python3.6/dist-packages/gensim/models/ldaseqmodel.py:290: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migr

In [98]:
#Printing Topics
ldaseq_cs_5.print_topics(time=0)

[[('would', 0.008684830898512193),
  ('data', 0.007333012750268088),
  ('|', 0.00722311289135918),
  ('type', 0.006747615276670389),
  ('function', 0.006462212336801802),
  ('one', 0.005850113805722877),
  ('like', 0.005824911861397692),
  ('use', 0.005594919501347261),
  ('used', 0.0041266680509089),
  ('code', 0.004109074712166421),
  ('programming', 0.004045279624803128),
  ('could', 0.003989145213782654),
  ('also', 0.003982871361903427),
  ('different', 0.0034331006517916815),
  ('might', 0.0034154715334401443),
  ('hash', 0.0031903984710954826),
  ('computer', 0.003187379601788174),
  ('many', 0.0031816503822783807),
  ('value', 0.0029851024378070782),
  ('way', 0.0029612607721715895)],
 [('graph', 0.013528951036760122),
  ('algorithm', 0.01278631411178872),
  ('problem', 0.012371058385478662),
  ('set', 0.011486805895084757),
  ('one', 0.010594770305180213),
  ('tree', 0.010272920394086763),
  ('find', 0.008942079810139034),
  ('node', 0.00870641845743941),
  ('number', 0.008321

In [99]:
ldaseq_cs_10.print_topics(time=0)

[[('computer', 0.010127846850077627),
  ('programming', 0.007187785299113789),
  ('model', 0.007164214029239413),
  ('would', 0.006337167795416569),
  ('also', 0.006258621914175983),
  ('different', 0.005701306904033286),
  ('many', 0.0056801073463390245),
  ('language', 0.005612432302551217),
  ('program', 0.005371076438380577),
  ('like', 0.005353548933336049),
  ('used', 0.005343585367194748),
  ('languages', 0.005253876787386317),
  ('one', 0.0051034109447592275),
  ('algorithms', 0.00477378896579824),
  ('use', 0.004713513471515479),
  ('may', 0.004652961747271503),
  ('system', 0.00453931687492437),
  ('could', 0.004363306945130367),
  ('much', 0.004205067894411252),
  ('theory', 0.004106592592985388)],
 [('time', 0.013378859645332567),
  ('hash', 0.010252685788235323),
  ('process', 0.00812005368897858),
  ('would', 0.00809569271948112),
  ('one', 0.007850974990903804),
  ('data', 0.007453060014484241),
  ('use', 0.007115465348251167),
  ('table', 0.006435995488019069),
  ('rand